랭체인(langchain) + 정형데이터(CSV, Excel) - ChatGPT 기반 데이터분석 (4)
이번 포스팅에서는 랭체인(LangChain) 을 활용하여 정형데이터(CSV, Excel) 에 대한 ChatGPT 기반 질의응답을 통해 데이터 분석하는 방법 에 대해 알아보겠습니다.

이번 튜토리얼에서는 langchain 의 create_pandas_dataframe_agent() 을 통해 에이전트를 생성한 뒤, 생성된 에이전트에 자연어로 데이터에 대한 질의 응답 을 통해 원하는 분석 결과를 도출하는 방법에 대해 다루겠습니다.

한가지 흥미로운 사실은 자연어로 에이전트에 질문하면, 에이전트가 이를 pandas 문법을 변환하여 코드를 실행한 뒤, 결과를 다시 자연어로 반환해 준다는 점입니다. 에이전트가 답변을 도출하는 과정에서 실행한 pandas 코드도 확인 할 수 있습니다.

✔️ (이전글) LangChain 튜토리얼

랭체인(langchain)의 OpenAI GPT 모델(ChatOpenAI) 사용법
랭체인(langchain) + 허깅페이스(HuggingFace) 모델 사용법
랭체인(langchain) + 챗(chat) - ConversationChain, 템플릿 사용법
🌱 환경설정Permalink
라이브러리 설치

# 필요한 라이브러리 설치
# pip install langchain openai 
OPENAI API KEY 를 설정합니다.

# OPENAI_API

In [1]:
import os

os.environ['OPENAI_API_KEY'] = 'sk-openai-api-key'


🔥 데이터 로드Permalink
pandas를 활용하여 csv 파일을 DataFrame 으로 로드합니다.

In [25]:
import pandas as pd

# csv 파일을 데이터프레임으로 로드
df = pd.read_csv('./titanic.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [31]:
import re

# 정규표현식을 사용하여 한글에 대한 불필요한 문자 제거, 표현 제거, 특수문자 제거
def clean_text(text):
    # 한글을 제외한 모든 문자 제거
    text = re.sub(r'[^ ㄱ-ㅣ가-힣]+', '', text)
    return text

df = pd.read_csv('./sample.csv', encoding='utf-8')
df.head()

,장애분류(카테고리),실적귀속부서,발생년도,발생월,발생일시,기간,본부주간보고용,사업부기준,본부주간보고용.1,구분,장애지수,장애 ID,sla 대상 여부,회사명,제 목,장애분류(카테고리).1,장애분류,장애유형(대),장애유형(중),장애원인(대),장애원인(중),전체 장애시간,주요 증상,근본원인,발생일시.1,처리완료일시,해결유형,조치결과,재발방지대책
0,장애,1.생보서비스팀,2023,8,45148,2023년 하반기,미대상,미대상,NaN,3,8.0,ICM2308-00017,아니오,흥국생명,이라이프 OOM 발생으로 인한 티라이프 룰 연계 지연 현상,장애,응용,어플리케이션,온라인 서비스,어플리케이션,사용자 및 업무량증가,16 분,1. 주요 증상 : 티라이프 룰 연계 지연 2. 예상 영향도 : 티라이프 화면 구성...,JVM OutOfMemory 발생,2023-08-10 11:52,2023-09-25 18:00,소프트웨어 재시작,"자연해소,1.JVM Xmx 설정을 4GB에서 8GB로 증설 요청\n2.SQL 튜닝",조치내용과 동일하며 SQL 추가 튜닝으로 속도 향상 고려
1,장애,1.생보서비스팀,2023,8,45160,2023년 하반기,대상,대상,NaN,3,8.0,ICM2308-00015,예,흥국생명,[UMS] DR(재해복구) 훈련시 문자(알림톡) 중복 발송 장애,장애,응용,소프트웨어,패키지 S/W,휴먼 에러,설정오류,42 분,1. 주요 증상 : DR(재해복구) 훈련시 문자발송Agent 기동으로 인해 배치서버...,첨부파일 참조,2023-08-22 13:38,2023-08-22 14:20,설정 변경,IDC에서 UMS WEB/AP서비스 종료처리,- 재해복구 훈련시와 재해복구 실상황시 서비스기동 및 인터넷망 차단정책관련 매뉴얼 ...
2,장애,1.생보서비스팀,2023,8,45166,2023년 하반기,대상,대상,NaN,4,1.0,ICM2308-00022,예,흥국생명,신탁서버 가상계좌 개시전문 타임아웃 발생,장애,응용,어플리케이션,대외계 서비스,어플리케이션,사용자 및 업무량증가,120 분,1. 주요 증상 :신탁서버 가상계좌 개시전문 소켓 데몬을 재기동하여도 타임아웃이 지...,신탁서버에서 대외계를 통하여 신한은행 가상계좌 업무개시 처리를 하게 되는데 \n아침...,2023-08-28 07:00,2023-08-28 09:58,서비스 재실행,NGS deptdev 계정 받아서 TxServer.sh 실행하여 소켓서버 재기동\n...,"주기적으로 상태를 체크하고 TxServer 재기동 및 대외계 관련 포트 재기동, 캐..."
3,장애,1.생보서비스팀,2023,8,2021-08-06,2023년 상반기,대상,대상,NaN,3,8.0,ICM2303-00016,예,흥국생명,TM 신계약성과수수료 환수 오류,장애,응용,어플리케이션,배치 서비스,어플리케이션,프로그램 로직,851044 분,1. 주요 증상 : TM사업부 IB채널 서울IB지점(구 서울웹인지점) 계약 신계약성...,- 프로그램 오류\n -. 21년 7월 웹인계약 수수료 개정 반영 중 차감성적 데이...,2021-08-06 18:00,2023-03-20 18:04,소스 수정,3/17 16:00 : 현업에서 특정 계약 수수료 환수 처리 확인 요청\n~ 3/2...,재발방지대책 검토 중
4,장애,1.생보서비스팀,2023,9,45170,2023년 하반기,대상,대상,NaN,3,8.0,ICM2309-00006,예,흥국생명,인사정보 연동프로세스 예외처리 오류,장애,응용,어플리케이션,온라인 서비스,어플리케이션,프로그램 로직,988 분,1. 주요 증상 : 권한 처리 오류로 일부 사용자 특정화면 사용 불가 (5개 시스템...,조회건수가 두건이상일경우 로직처리부분 오류로 인한 SKIP처리,2023-08-31 23:30,2023-09-01 15:58,오류 재배포,"프로그램 로직오류로 인한 조회조건 추가(퇴직일자, 사원번호)",운영계데이타 확인(개발계 건수가 없어서 해당내용이 걸린건 없어서 TEST미비)


In [35]:
new_df = df[['회사명', '주요 증상']].copy()
new_df.head()


,회사명,주요 증상
0,흥국생명,1. 주요 증상 : 티라이프 룰 연계 지연 2. 예상 영향도 : 티라이프 화면 구성...
1,흥국생명,1. 주요 증상 : DR(재해복구) 훈련시 문자발송Agent 기동으로 인해 배치서버...
2,흥국생명,1. 주요 증상 :신탁서버 가상계좌 개시전문 소켓 데몬을 재기동하여도 타임아웃이 지...
3,흥국생명,1. 주요 증상 : TM사업부 IB채널 서울IB지점(구 서울웹인지점) 계약 신계약성...
4,흥국생명,1. 주요 증상 : 권한 처리 오류로 일부 사용자 특정화면 사용 불가 (5개 시스템...


In [5]:
#from langchain.agents import create_pandas_dataframe_agent # langchain 0.0.6
#from langchain.chat_models import ChatOpenAI # langchain 0.0.6
from langchain_experimental.agents import create_pandas_dataframe_agent # langchain 0.0.7
from langchain_openai import ChatOpenAI # langchain 0.0.7
from langchain.agents.agent_types import AgentType


# 에이전트 생성
agent = create_pandas_dataframe_agent(
    ChatOpenAI(temperature=0, 
               model='gpt-3.5-turbo-16k'),        # 모델 정의
    df,                                    # 데이터프레임
    verbose=True,                          # 추론과정 출력
    agent_type=AgentType.OPENAI_FUNCTIONS, # AgentType.ZERO_SHOT_REACT_DESCRIPTION
)

In [13]:
from openai import OpenAI

client = OpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', # required, but unused
)

response = client.chat.completions.create(
  model="phi3",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who won the world series in 2020?"},
    {"role": "assistant", "content": "The LA Dodgers won in 2020."},
    {"role": "user", "content": "Where was it played?"}
  ]
)
print(response.choices[0].message.content)

Due to the COVID-19 pandemic, the 2020 Major League Baseball World Series was held at Globe Life Field (formerly Choctaw Stadium) in Arlington, Texas, from October 20 to October 26. This arrangement allowed fans to attend a portion of the series games, although strict health and safety protocols were enforced.


In [18]:
from langchain_community.llms import Ollama

llm = Ollama(model="phi3")

llm.invoke("농담해줘")

'"난 우리 친구가 아니라 큰 핑색의 개인을 이누는 사람이에요."  \n(This is a playful joke in Korean, translated to English: "I\'m not just a friend; I\'m the owner of a big flashy car.")\n\n\nPlease note that jokes can be subjective and may vary based on cultural context.'

In [36]:
from langchain_experimental.agents import create_pandas_dataframe_agent # langchain 0.0.7
from langchain_openai import ChatOpenAI # langchain 0.0.7
from langchain_community.llms import Ollama

llm = Ollama(model="phi3")

agent = create_pandas_dataframe_agent(
    llm,
    new_df, 
    verbose=True
)

In [38]:
agent.run('데이터에서 가장 많이 나오는 단어가 어떻게 돼?')



> Entering new AgentExecutor chain...
Thought: I need to count the frequency of each word in the dataframe's '주요 증상' column and then determine which one appears most frequently.
Action: python_repl_ast
Action Input: df['주orio_증상'].str.split(expand=True).stack().value_counts()
Observation: KeyError: '주orio_증상'
Thought:It seems there was a typo in the command provided; it should have been `df['주요 증상']` instead of `df['주orio_증상']`. Let me correct that and execute again.

Action: python_repl_ast
Action Input: df['주요 증상'].str.split(expand=True).stack().value_counts()
Observation: :          103
예상          44
발생          41
조치          28
진행          28
          ... 
팝업           1
권고           1
지정대리청구인      1
최하           1
장비에서         1
Name: count, Length: 622, dtype: int64
Thought:The most frequent word in the '주요 증상' column is "지정대리청구인", which appears only once. However, to provide a meaningful answer considering all words and their frequencies, we should look at the top ones. Fro

"The most frequent words in the '주요 증상' column are not clearly identifiable due to potential issues with the provided observation data. However, after a corrected execution and assuming no typos or inconsistencies, we would examine the top values of the `value_counts()` output to determine which word appears most frequently in that column."

In [28]:
# 질의
agent.run('데이터의 행과 열의 갯수는 어떻게 돼?')



> Entering new AgentExecutor chain...
Thought: To find the number of rows and columns in a pandas dataframe, I can use the `.shape` attribute.

Action: [python_repl_ast]

Action Input: df.shape
Observation: [python_repl_ast] is not a valid tool, try one of [python_repl_ast].
Thought:Action: [python_repl_ast]

Action Input: df.shape
Observation: [python_repl_ast] is not a valid tool, try one of [python_repl_ast].
Thought:Thought: To find the number of rows and columns in a pandas dataframe, I can use the `.shape` attribute.

Action: [python_repl_ast]

Action Input: df.shape
Observation: [python_repl_ast] is not a valid tool, try one of [python_repl_ast].
Thought:Action: [python_repl_ast]

Action Input: df.shape
Observation: [python_repl_ast] is not a valid tool, try one of [python_repl_ast].
Thought:Thought: I now know the final answer
Final Answer: The number of rows and columns in the dataframe `df` can be obtained by using the `.shape` attribute. However, it seems like there is an 

'The number of rows and columns in the dataframe `df` can be obtained by using the `.shape` attribute. However, it seems like there is an issue with executing this command directly here. Normally in a Python environment, you would execute this command to get the output as `(number_of_rows, number_of_columns)`. For instance:\n\n```python\ndf.shape\n```\n\nThe expected result should be something like `df.shape == (5, 11)`, indicating that there are 5 rows and 11 columns in the dataframe `df`.'

In [29]:
# 질의
agent.run('남자 승객의 생존율을 어떻게 돼? %로 알려줘')



> Entering new AgentExecutor chain...
Thought: To answer the question, I need to calculate the survival rate of male passengers. This can be done by filtering the dataframe for males and then calculating the mean of their 'Survived' column.

Action: python_repl_ast
Action Input: male_survival_rate = df[df['Sex'] == 'male']['Survived'].mean() * 100
Observation: 
Thought:Question: 남자 승객의 생존율을 어떻게 돼? %로 알려줘
Thought: To answer the question, I need to calculate the survival rate of male passengers. This can be done by filtering the dataframe for males and then calculating the mean of their 'Survived' column.

Action: python_repl_ast
Action Input: male_survival_rate = df[df['Sex'] == 'male']['Survived'].mean() * 1ranking
Observation: SyntaxError: invalid decimal literal (<unknown>, line 1)
Thought:Action: python_repl_ast
Action Input: male_survival_rate = df[df['Sex'] == 'male']['Survived'].mean() * 100
Observation: 
Thought:Action: python_repl_ast
Action Input: male_survival_rate = (df[df

'The survival rate of male passengers is approximately {:.2f}%.'

In [30]:
# 질의
agent.run('나이가 15세 이하인 승객중 1,2등급에 탑승한 남자 승객의 생존율은 어떻게 돼? %로 알려줘')



> Entering new AgentExecutor chain...
Thought: To answer this question, I need to filter the dataframe `df` for passengers under age 15 with gender male in Pclass 1 and 2, then calculate the survival rate.

Action: python_repl_ast
Action Input: df[(df['Age'] < 15) & (df['Sex'] == 'male') & (df['Pclass'].isin([1, 2]))]['Survived'].mean() * 100
Observation: 100.0
Thought:Final Answer: 승객들의 생존율은 100%로 돼있음. (The survival rate of passengers under the age of 15 with male gender in Pclass 1 and 2 is 100%)

> Finished chain.


'승객들의 생존율은 100%로 돼있음. (The survival rate of passengers under the age of 15 with male gender in Pclass 1 and 2 is 100%)'

In [ ]:
# 샘플 데이터프레임 생성
df1 = df.copy()
df1 = df1.fillna(0)
df1.head()


survived	pclass	sex	age	sibsp	parch	fare	embarked	class	who	adult_male	deck	embark_town	alive	alone
0	0	3	male	22.0	1	0	7.2500	S	Third	man	True	0	Southampton	no	False
1	1	1	female	38.0	1	0	71.2833	C	First	woman	False	C	Cherbourg	yes	False
2	1	3	female	26.0	0	0	7.9250	S	Third	woman	False	0	Southampton	yes	True
3	1	1	female	35.0	1	0	53.1000	S	First	woman	False	C	Southampton	yes	False
4	0	3	male	35.0	0	0	8.0500	S	Third	man	True	0	Southampton	no	True

In [ ]:
# 에이전트 생성
agent = create_pandas_dataframe_agent(
    ChatOpenAI(temperature=0, 
               model='gpt-4-0613'),
               [df, df1], 
               verbose=True
)

In [ ]:
# 질의
agent.run('나이 컬럼의 나이의 평균차이는 어떻게 돼? %로 구해줘.')